In [ ]:
import numpy as np
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
train = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/train.csv')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/sample_submission.csv')
test.head()

In [ ]:
print("Кол-во классов: {}".format(train['emotion'].nunique()))

In [ ]:
BATCH_SIZE = 128
IMAGE_SIZE = 48

image_gen = ImageDataGenerator() 

train_generator = image_gen.flow_from_directory(directory=str('/kaggle/input/train-data-plus-plus-v3'),
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                class_mode='categorical')

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax'),
    ])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
%%timeit -n 10 -r 10

test = model(np.random.rand(1,48,48,3).astype(np.float32))

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_generator, epochs=100, batch_size=BATCH_SIZE)

In [ ]:
predict_dict = {0:'anger',1:'contempt',2:'disgust',3:'fear',4:'happy',5:'neutral',6:'sad',7:'surprise',8:'uncertain'}

In [ ]:
test_data = tf.keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/test-data',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE,IMAGE_SIZE)
)

In [ ]:
emotion = model.predict(test_data)

In [ ]:
emotion_cm = [predict_dict[np.argmax(i)] for i in emotion]

In [ ]:
test['emotion'] = emotion_cm
test.head()

In [ ]:
test.to_csv('/kaggle/working/submission.csv', sep=',', index=False)

В этом ноутбуке я использовал первую модель, но при этом увеличил датасет в 2 раза. 
Я попытался адаптировать метод использованный в этом курсе для серых картинок:

import os

from PIL import Image

import random

from os import listdir

import numpy as np

for i1 in listdir('D://Загрузки//train//train'):

    for i2 in listdir('D://Загрузки//train//train//'+i1):
    
        img = Image.open('D://Загрузки//train//train//'+i1+'//'+i2)
        img = np.array(img)

        shape = img.shape
        sh = shape[0]
        sw = shape[1]

        dh = sh//4 * random.choice([-1, 1])
        dw = sw//4 * random.choice([-1, 1])

        img_zero =  np.array(Image.fromarray(np.zeros(shape), 'RGB'))

        img_zero[ max(-dh,0):min(sh-dh,sh), max(-dw,0):min(sw-dw, sw), :] = img[max( dh,0):min(sh+dh,sh), max( dw,0):min(sw+dw, sw), :]


        img = Image.fromarray(img_zero, 'RGB')
        image = img.save("D://Загрузки//train//train//"+i1+'//+'+i2)
 
 Но при этом увеличил кол-во эпох до 100, что привело к переобучению. Score на обучающих данных: 0.4333